In [1]:
import numpy as np
import videoflow
import videoflow.core.flow as flow
from videoflow.consumers import VideofileWriter, CommandlineConsumer
from videoflow.producers import VideofileReader
from videoflow_contrib.processors.detector_tf import TensorflowObjectDetector
from videoflow_contrib.processors.tracker_sort import KalmanFilterBoundingBoxTracker
from videoflow.processors.vision.annotators import TrackerAnnotator, BoundingBoxAnnotator


In [2]:
class BoundingBoxesFilter(videoflow.core.node.ProcessorNode):
    def __init__(self, class_indexes_to_keep):
        self._class_indexes_to_keep = class_indexes_to_keep
        super(BoundingBoxesFilter, self).__init__()
    
    def process(self, dets):
        '''
        Keeps only the boxes with the class indexes
        specified in self._class_indexes_to_keep

        - Arguments:
            - dets: np.array of shape (nb_boxes, 6) \
                Specifically (nb_boxes, [xmin, ymin, xmax, ymax, class_index, score])
        '''
        f = np.array([dets[:, 4] == a for a in self._class_indexes_to_keep])
        f = np.any(f, axis = 0)
        filtered = dets[f]
        return filtered

def run_flow():
    
    tensorflow_model_path = "/Users/dearj019/Downloads/faster_rcnn_resnet101_coco_2018_01_28/frozen_inference_graph.pb"
    tensorflow_model_classes = 90
    output_file = "output.avi"
    output_file2 = "output2.avi"
    input_file = "/Users/dearj019/Downloads/disney2.mp4"
    class_labels_path = "mscoco_labels.pbtxt"
    
    reader = VideofileReader(input_file, 2)
    detector = TensorflowObjectDetector(tensorflow_model_path, tensorflow_model_classes)(reader)
    detector_annotator = BoundingBoxAnnotator(class_labels_path)(reader, detector)
    tracker = KalmanFilterBoundingBoxTracker()(detector)
    tracker_printer = CommandlineConsumer()(tracker)
    annotator = TrackerAnnotator()(reader, tracker)
    writer = VideofileWriter(output_file, fps = 30)(annotator)
    writer1 = VideofileWriter(output_file2, fps = 30)(detector_annotator)
    fl = flow.Flow([reader], [writer], flow_type = flow.BATCH)
    fl.run()
    fl.join()

In [3]:
run_flow()

2019-05-16 16:33:56,141 - videoflow.core - INFO - Started task allocation for 8 tasks
2019-05-16 16:33:56,183 - videoflow.core - INFO - Started running flow.


[[4.49908754e+02 8.94678081e-07 6.28376891e+02 1.41900451e+02
  2.00000000e+00]
 [3.17394592e+02 5.27391052e+02 5.92527100e+02 7.90220032e+02
  1.00000000e+00]]
